In [18]:
%matplotlib inline
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [19]:
data = pd.read_csv(r"/Volumes/D/py-workspace/PFE/dataset/Input/outband_QoS_ITU-QoE.csv", nrows=20000)
#data = pd.read_csv(r"/Volumes/D/py-workspace/PFE/dataset/Input/outband_QoS_ITU-QoE.csv")
data = data.dropna()

print(data.columns.values)
# Map Continuous target to class
# # Import numpy, generate the feature matrix and the output y
X = data[['outbandQoS_DL_TP (kbps)', 'outbandQoS_UL_TP (kbps)', 'outbandQoS_RTT (ms)', 'outbandQoS_LOSS (ratio)']].values
y = data[['QoE_ITU_JT_046_VP9_1280x780']].values

# Print the first row
print("X: ", X[0:5, :])

['outbandQoS_DL_TP (kbps)' 'outbandQoS_UL_TP (kbps)' 'outbandQoS_RTT (ms)'
 'outbandQoS_RTT_STD (ms)' 'outbandQoS_LOSS (ratio)'
 'QoE_ITU_JT_046_VP9_1280x780']
X:  [[2.21530127e+03 1.05268433e+02 5.54525805e+00 3.65501880e-03]
 [5.49837451e+03 1.38147998e+03 4.67492142e+01 3.75949960e-03]
 [1.04619312e+03 9.26093521e+01 2.83280048e+01 2.02680659e-02]
 [2.08757959e+03 6.56152466e+02 4.22402802e+01 1.31945405e-02]
 [9.26885681e+02 4.39348297e+02 9.77033997e+01 8.94861670e-03]]


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

def rfr_model(X, y):
    # Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

    y = np.ravel(y)
    grid_result = gsc.fit(X, y)
    
    best_params = grid_result.best_params_

    rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"], random_state=False, verbose=False)
    # Perform K-Fold CV
    scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')
    #predictions = cross_val_predict(rfr, X, y, cv=10)
    
    #scores = None
    predictions = None
    return scores, predictions

#predictions = cross_val_predict(rfr_model, X, y, cv=10)

scores, predictions = rfr_model(X, y)

display(scores)
#display(predictions)

# df1 = pd.DataFrame(y) 
# df = pd.DataFrame(predictions)
# df1['pred'] = df.iloc[:,0]

# filename = 'itu_rf_reg_pred.csv'
# df1.to_csv(filename)

array([-0.33749783, -0.33404952, -0.32520722, -0.33489578, -0.33121301,
       -0.33966687, -0.32596874, -0.32584136, -0.32636478, -0.3359489 ])